### This file contains how to download and use MobileNet-V1 with examples
- Download MobileNet-V1 model from  http://download.tensorflow.org/models/mobilenet_v1_2018_08_02/mobilenet_v1_1.0_224.tgz
- Download MobileNet-V2 model from https://storage.googleapis.com/mobilenet_v2/checkpoints/mobilenet_v2_1.0_224.tgz
- Download MobileNet-SSD V2 at http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz



In [37]:

import numpy as np
import cv2
import os, time
import tensorflow as tf
from tensorflow.python.keras.utils.data_utils import get_file

tf.python.enable_eager_execution = False
np.random.seed(123)

class Detector:
    def __init__(self):
        pass

    def readClasses(self, classesFilePath):
        with open(classesFilePath, 'r') as f:
            self.classlist = f.read().split()

        #colorList
        self.colorList = np.random.uniform(low=0, high=255, size=(len(self.classlist),3))

        print(len(self.classlist), len(self.colorList))

    def downloadModel(self, modelURL):
        fileName = os.path.basename(modelURL)
        self.modelName =  fileName[:fileName.index('.')]
        print(fileName)
        print(self.modelName)
        self.cacheDir = "./pretrained_models"
        os.makedirs(self.cacheDir, exist_ok=True)
        get_file(fname=fileName, origin=modelURL, cache_dir=self.cacheDir, cache_subdir="mobilenet_ssd_v2", extract=True)

    def loadModel(self):
        print("Loading MobileNet SSD Object Model" + self.modelName)
        tf.keras.backend.clear_session()
        self.model = tf.saved_model.load(os.path.join(self.cacheDir,"mobilenet_ssd_v2",self.modelName,"saved_model"))
        print("Model "+self.modelName+" Loaded Successfully")

    def createBoundingBox(self, image, threshold):
        inputTensor = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        inputTensor = tf.convert_to_tensor(inputTensor)
        inputTensor = inputTensor[tf.newaxis,...]
        detections = self.model(inputTensor)
        bboxes = detections['detection_boxes'][0]
        classIndexes = detections['detection_classes'][0].numpy().astype(np.int32)
        classScores = detections['detection_scores'][0].numpy()
        imH, imW, imC = image.shape

        bboxIdx = tf.image.non_max_suppression(bboxes, classScores, max_output_size=50, iou_threshold=0.5, score_threshold= threshold)

        if len(bboxes)!= 0:
            for i in range(0, len(bboxes)):
                bbox = tuple(bboxes[i].numpy())
                classConfidence = round(100*classScores[i])
                classIndex = classIndexes[i]

                classLabelText = self.classlist[classIndex].upper()
                classColor = self.colorList[classIndex]

                displayText = f'{classLabelText}: {classConfidence}'

                ymin, xmin, ymax, xmax = bbox
                xmin, xmax, ymin, ymax = (int(xmin*imW), int(xmax*imW), int(ymin*imH), int(ymax*imH))

                cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color = classColor, thickness = 1)
                cv2.putText(image, displayText, (xmin, ymin-10), cv2.FONT_HERSHEY_TRIPLEX, 1, classColor, 2)
                lineWidth = min(int((xmax-xmin)*0.2), int((ymax-ymin)*0.2))
                cv2.line(image, (xmin,ymin), (xmin+lineWidth, ymin), classColor, thickness=5)
                cv2.line(image, (xmin,ymin), (xmin, ymin+lineWidth), classColor, thickness=5)
                cv2.line(image, (xmax,ymin), (xmax-lineWidth, ymin), classColor, thickness=5)
                cv2.line(image, (xmax,ymin), (xmax, ymin+lineWidth), classColor, thickness=5)

                cv2.line(image, (xmin,ymax), (xmin+lineWidth, ymax), classColor, thickness=5)
                cv2.line(image, (xmin,ymax), (xmin, ymax-lineWidth), classColor, thickness=5)
                cv2.line(image, (xmax,ymax), (xmax-lineWidth, ymin), classColor, thickness=5)
                cv2.line(image, (xmax,ymax), (xmax, ymin-lineWidth), classColor, thickness=5)
        return image


    def predictImage(self, imagePath, threshold=0.5):
        image = cv2.imread(imagePath)
        bboxImage = self.createBoundingBox(image, threshold)
        cv2.namedWindow("Result", cv2.WINDOW_NORMAL)
        cv2.imshow("Result", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    def predictVideo(self, videoPath, threshold=0.5):
        cap = cv2.VideoCapture(videoPath)

        if (cap.isOpened()==False):
            print("Error Opening FIle...")
            return
        (success, image) = cap.read()
        startTime=0
        cv2.namedWindow("Result", cv2.WINDOW_NORMAL)
        while success:
            currentTime = time.time()
            fps = 1/(currentTime-startTime)
            startTime = currentTime
            bboxImage = self.createBoundingBox(image=image, threshold=threshold)
            cv2.putText(bboxImage, "FPS: " + str(int(fps)), (20,70), cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0), 2)
            cv2.imshow("Result", bboxImage)
            key = cv2.waitKey(1) & 0xFF
            if key==ord("q"):
                break
            (success, image) = cap.read()

        cv2.destroyAllWindows()

        





    


In [38]:
detector = Detector()

classFiles = "coco.names"
detector.readClasses(classesFilePath= classFiles)

modelURL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz"
imagePath = "C:\\Users\\F85SJ00\\Documents\\Python-Scripts\\0_MlM8CnEX2hqpziyI.jpg"
threshold = 0.5
videoPath = 0

detector.downloadModel(modelURL=modelURL)
detector.loadModel()
detector.predictImage(imagePath=imagePath, threshold=threshold)
#detector.predictVideo(videoPath=videoPath, threshold=threshold)



94 94
ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz
ssd_mobilenet_v2_320x320_coco17_tpu-8
Loading MobileNet SSD Object Modelssd_mobilenet_v2_320x320_coco17_tpu-8
Model ssd_mobilenet_v2_320x320_coco17_tpu-8 Loaded Successfully
Error Opening FIle...
